# **Step1_AI면접관 Agent v1.0**

## **1. 환경준비**

### (1) 구글 드라이브

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project_genai)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### (2) 라이브러리

In [2]:
!pip install -r /content/drive/MyDrive/KT_AIVLE/05.MINI2/requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

### (3) OpenAI API Key 확인
* api_key.txt 파일에 다음의 키를 등록하세요.
    * OPENAI_API_KEY
    * NGROK_AUTHTOKEN

In [3]:
import os

def load_api_keys(filepath="api_key.txt"):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                os.environ[key.strip()] = value.strip()

path ='/content/drive/MyDrive/KT_AIVLE/05.MINI2/'
# API 키 로드 및 환경변수 설정
load_api_keys(path + 'api_key.txt')

In [4]:
print(os.environ['OPENAI_API_KEY'][:30])

sk-proj-rFBDwtWMLgSipbzOUgDYj2


## **2. App.py**

* 아래 코드에, Step1 혹은 고도화 된 Step2 파일 코드를 붙인다.
    * 라이브러리
    * 함수들과 그래프
* Gradio 코드는 그대로 사용하거나 일부 수정 가능

In [13]:
%%writefile app.py

####### 여러분의 함수와 클래스를 모드 여기에 붙여 넣읍시다. #######
## 1. 라이브러리 로딩 ---------------------------------------------
import pandas as pd
import numpy as np
import os
import openai
import random
import ast
import fitz
from docx import Document

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from typing import Annotated, Literal, Sequence, TypedDict, List, Dict
from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.output_parsers import CommaSeparatedListOutputParser
from langgraph.graph import StateGraph, END

## ---------------- 1단계 : 사전준비 ----------------------

# 1) 파일 입력 --------------------
def extract_text_from_file(file_path: str) -> str:
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".pdf":
        doc = fitz.open(file_path)
        text = "\n".join(page.get_text() for page in doc)
        doc.close()
        return text
    elif ext == ".docx":
        doc = Document(file_path)
        return "\n".join(p.text for p in doc.paragraphs if p.text.strip())
    else:
        raise ValueError("지원하지 않는 파일 형식입니다. PDF 또는 DOCX만 허용됩니다.")

# 2) State 선언 --------------------
class InterviewState(TypedDict):
    # 고정 정보
    resume_text: str
    resume_summary: str
    resume_keywords: List[str]
    question_strategy: Dict[str, Dict]

    # 인터뷰 로그
    current_question: str
    current_answer: str
    current_strategy: str
    conversation: List[Dict[str, str]]
    evaluation : List[Dict[str, str]]
    next_step : str

# 3) resume 분석 --------------------
def analyze_resume(state: InterviewState) -> InterviewState:
    resume_text = state.get("resume_text", "")
    if not resume_text:
        raise ValueError("resume_text가 비어 있습니다. 먼저 텍스트를 추출해야 합니다.")

    llm = ChatOpenAI(model="gpt-4o-mini")

    # 요약 프롬프트 구성
    summary_prompt = ChatPromptTemplate.from_template(
        '''당신은 이력서를 바탕으로 인터뷰 질문을 설계하는 AI입니다.
        다음 이력서 및 자기소개서 내용에서 질문을 뽑기 위한 중요한 내용을 10문장 정도로 요약을 해줘(요약시 ** 기호는 사용하지 말것):\n\n{resume_text}'''
    )
    formatted_summary_prompt = summary_prompt.format(resume_text=resume_text)
    summary_response = llm.invoke(formatted_summary_prompt)
    resume_summary = summary_response.content.strip()

    # 키워드 추출 프롬프트 구성
    keyword_prompt = ChatPromptTemplate.from_template(
        '''당신은 이력서를 바탕으로 인터뷰 질문을 설계하는 AI입니다.
        다음 이력서 및 자기소개서내용에서 질문을 뽑기 위한 중요한 핵심 키워드를 5~10개 추출해줘. 도출한 핵심 키워드만 쉼표로 구분해줘:\n\n{resume_text}'''
    )
    formatted_keyword_prompt = keyword_prompt.format(resume_text=resume_text)
    keyword_response = llm.invoke(formatted_keyword_prompt)

    parser = CommaSeparatedListOutputParser()
    resume_keywords = parser.parse(keyword_response.content)

    return {
        **state,
        "resume_summary": resume_summary,
        "resume_keywords": resume_keywords,
    }

# 4) 질문 전략 수립 --------------------

def generate_question_strategy(state: InterviewState) -> InterviewState:
    resume_summary = state.get("resume_summary", "")
    resume_keywords = ", ".join(state.get("resume_keywords", []))

    prompt = ChatPromptTemplate.from_template("""
당신은 이력서를 바탕으로 인터뷰 질문을 설계하는 AI입니다.
다음 이력서 요약과 키워드를 기반으로, 다음 3가지 질문 부문 별로 질문 방향과 예시 질문 2개를 딕셔너리 형태로 출력해줘.

- 이력서 요약:
{resume_summary}

- 이력서 키워드:
{resume_keywords}

아래 형식을 따라줘:
딕셔너리 형태에서 key는 3가지 질문 부문이야. '경력 및 경험', '동기 및 커뮤니케이션', '논리적 사고'
각 key에 해당하는 value는 딕셔너리 형태로 여기의 key는 2가지로 '질문전략', '예시질문' 이야.
'질문전략'의 value는 각 질문 부문에 대해서, 이력서요약 및 키워드로부터 질문 전략 및 방향을 결정하는 구체적인 문장이야.
'예시질문'의 value는 리스트형태로, 질문전략에 맞는 구체적인 예시 질문 2개 문장이야.

[예시]
{{{{ "경력 및 경험": {{'질문전략' : "지원자의 실무 경험, 기술적 능력 및 프로젝트 관리 경험을 중점적으로 평가하는 방향으로 질문을 구성합니다. 이를 통해 지원자가 과거에 경험한 기술적 도전과 문제 해결 방식, 팀 내의 역할 등을 확인할 수 있습니다.",
'예시질문': ['KT의 AI 연구소에서 인턴으로 근무하며 OCR 기반 문서 처리 시스템을 고도화할 때 겪었던 기술적 난관은 무엇이었고, 이를 어떻게 극복했는지 설명해 주시겠습니까?',
'빅데이터 학생연합에서 프로젝트를 리드했던 경험에 대해 이야기해 주세요. 어떤 과제가 있었고, 팀원들과의 소통 과정에서 어려움은 없었는지 궁금합니다.']}},
"동기 및 커뮤니케이션": ...,
"논리적 사고": ...
}}}}
앞뒤로 ```python ~ ``` 붙이는것은 하지 마.
""")

    llm = ChatOpenAI(model="gpt-4o-mini")
    formatted_prompt = prompt.format(resume_summary=resume_summary, resume_keywords=resume_keywords)
    response = llm.invoke(formatted_prompt)

    # 딕셔너리로 변환
    dict_value = response.content.strip()
    if isinstance(dict_value, str):
        try:
            strategy_dict = ast.literal_eval(dict_value)
        except Exception as e:
            raise ValueError("question_strategy를 딕셔너리로 변환하는 데 실패했습니다.") from e

    return {
        **state,
        "question_strategy": strategy_dict
    }

# 5) 1단계 하나로 묶기 --------------------

def preProcessing_Interview(file_path: str) -> InterviewState:
    # 파일 입력
    resume_text = extract_text_from_file(file_path)

    # state 초기화
    initial_state: InterviewState = {
        "resume_text": resume_text,
        "resume_summary": '',
        "resume_keywords": [],
        "question_strategy": {},

        "current_question": '',
        "current_answer": '',
        "current_strategy": '',
        "conversation": [],
        "evaluation": [],
        "next_step" : ''
    }

    # Resume 분석
    state = analyze_resume(initial_state)

    # 질문 전략 수립
    state = generate_question_strategy(state)

    # 첫번째 질문 생성
    question_strategy = state["question_strategy"]
    example_questions = question_strategy["경력 및 경험"]["예시질문"]
    selected_question = random.choice(example_questions)

    return {
            **state,
            "current_question": selected_question,
            "current_strategy": "경력 및 경험"
            }


## ---------------- 2단계 : 면접 Agent ----------------------

# 1) 답변 입력 --------------------
def update_current_answer(state: InterviewState, user_answer: str) -> InterviewState:
    return {
        **state,
        "current_answer": user_answer.strip()
    }

# 2) 답변 평가 --------------------
def evaluate_answer(state: InterviewState) -> InterviewState:
    question = state["current_question"]
    answer = state["current_answer"]
    current_topic = state.get("current_strategy", "기타")



    # 프롬프트 구성
    prompt_template = PromptTemplate(
        input_variables=["question", "answer"],
        template="""

당신은 구조화된 기준에 따라 지원자의 면접 답변을 평가하는 AI 평가자입니다.
아래 질문과 답변을 읽고, 각 평가 항목별로 ‘상’, ‘중’, ‘하’ 중 하나를 선택하고 이유를 함께 제시해 주세요.

[질문]
{question}

[답변]
{answer}

아래 다섯 가지 항목에 대해 각 항목별로 ‘상’, ‘중’, ‘하’ 중 하나로 평가해 주세요.
각 등급에 대한 간단한 이유도 함께 적어 주세요.

1. 질문과의 연관성 – 답변이 질문의 핵심 의도에 어느 정도 부합했는지 평가해 주세요.
(예: 질문의 키워드를 잘 파악했는지, 전혀 엉뚱한 방향으로 흐르지 않았는지 등)
상: 질문의 핵심 의도에 정확히 부합하며, 답변이 전반적으로 질문에 집중되어 있음

중: 질문과 관련은 있으나, 핵심 포인트가 부족하거나 방향이 다소 벗어남

하: 질문과 거의 관련이 없거나, 엉뚱한 주제를 중심으로 전개됨



2.답변의 구체성 –  단순한 주장이나 포부보다는 구체적인 경험, 수치, 상황, 역할 등이 포함되어 있는지 확인해 주세요.
(예: "노력했습니다"보다는 "어떤 방식으로 노력했는지"가 드러나야 함)
상: 구체적인 경험, 수치, 상황, 역할 등이 잘 드러나 있음 (ex. 프로젝트 내용, 성과 수치, 팀 내 역할 등)

중: 주장이나 포부는 있지만, 경험이나 사례가 부족하거나 모호함

하: 단순한 추상적 표현만 있고, 경험이나 실제 내용은 거의 없음



3. 답변의 길이 – 답변이 충분한 분량으로 작성되었는지
상: 핵심 내용이 잘 담긴 충분한 길이의 문장 (예: 3~5문장 이상, 정보 밀도 높음)

중: 다소 짧거나 내용이 압축적이긴 하나 요지는 전달됨

하: 한두 문장으로 끝나며, 정보량이 현저히 부족함



4. 답변의 태도 – 글을 통해 진정성, 책임감, 적극성, 성찰적 태도 등이 전달되는지 평가해 주세요.
(예: 실패 경험에 대해 남 탓이 아닌 본인의 개선점을 중심으로 서술했다면 긍정적으로 평가)
상: 진정성, 책임감, 성찰적 태도가 드러나며, 신뢰감 있는 어조

중: 기본적인 태도는 갖추었으나 감정이나 책임감이 덜 전달됨

하: 무성의하거나 회피성 표현이 많고, 책임 회피성 응답이 포함됨


5. 문장 완성도 – 전체 문장이 논리적으로 잘 연결되었는지, 줄임말이나 비문 없이 자연스럽고 읽기 쉬운 문장으로 구성되었는지 확인해 주세요.
상: 문장이 자연스럽고 논리적이며, 비문·줄임말 없이 깔끔하게 연결됨

중: 문장은 전반적으로 이해되나, 일부 어색하거나 비문이 있음

하: 문장이 비정상적이거나 줄임말·은어·축약어가 많아 가독성이 떨어짐


[응답 형식]
{{
  "질문과의 연관성": {{"등급": "상|중|하", "이유": "…"}},
  "답변의 구체성": {{"등급": "상|중|하", "이유": "…"}},
  "답변의 길이": {{"등급": "상|중|하", "이유": "…"}},
  "답변의 태도": {{"등급": "상|중|하", "이유": "…"}},
  "문장 완성도": {{"등급": "상|중|하", "이유": "…"}}
}}
"""
    )

    # prompt_text = prompt_template.format(question=question, answer=answer)
    # response = llm([HumanMessage(content=prompt_text)]).content

    # updated_conversation = state.get("conversation", []) + [
    #     {"question": question, "answer": answer}
    # ]
    # updated_evaluation = state.get("evaluation", []) + [response.strip()]

     # LLM 호출
    llm = ChatOpenAI(model="gpt-4o-mini")
    prompt_text = prompt_template.format(question=question, answer=answer)
    response = llm([HumanMessage(content=prompt_text)]).content

    # topic 필드를 평가 문자열에 직접 삽입
    response_text = response.strip()
    if response_text.endswith("}"):
        response_text_with_topic = response_text[:-1] + f', "topic": "{current_topic}"' + "}"
    else:
        response_text_with_topic = response_text  # fallback

    # 대화 및 평가 누적
    updated_conversation = state.get("conversation", []) + [{"question": question, "answer": answer}]
    updated_evaluation = state.get("evaluation", []) + [response_text_with_topic]

    return {
        **state,
        "evaluation": updated_evaluation,
        "conversation": updated_conversation
    }


# 3) 인터뷰 진행 검토 --------------------
def decide_next_step(state: InterviewState) -> InterviewState:
    evaluations = state.get("evaluation", [])
    strategy = state.get("question_strategy", {})

    if not evaluations:
        print("🟡 [DEBUG] 평가 없음 → generate")
        return {**state, "next_step": "generate"}

    latest_eval_raw = evaluations[-1]
    try:
        latest_eval = eval(latest_eval_raw)
    except Exception as e:
        print("❌ [DEBUG] 평가 파싱 실패:", e)
        return {**state, "next_step": "end"}

    current_topic = latest_eval.get("topic", None)
    topic_list = list(strategy.keys())

    if current_topic not in topic_list:
        print(f"⚠️ [DEBUG] 현재 topic '{current_topic}'이 topic_list에 없음 → 종료")
        return {**state, "next_step": "end"}

    current_index = topic_list.index(current_topic)
    topic_attempts = [e for e in evaluations if current_topic in e]
    attempt_count = len(topic_attempts)

    # 최근 평가에서 '하' 포함 여부
    low_fields = ["답변의 길이", "답변의 태도", "문장 완성도"]
    is_low = any(
        latest_eval.get(field, {}).get("등급") == "하"
        for field in low_fields
    )

    # # 📌 디버그 출력
    # print(f"\n🧠 [DEBUG] 현재 topic: {current_topic}")
    # print(f"🗂️ [DEBUG] topic_list: {topic_list}")
    # print(f"📍 [DEBUG] current_index: {current_index}")
    # print(f"🔁 [DEBUG] 시도 횟수: {attempt_count}")
    # print(f"⚠️ [DEBUG] 최근 평가 is_low: {is_low}")

    # 결정 로직
    if attempt_count >= 2:
        if current_index + 1 < len(topic_list):
            next_topic = topic_list[current_index + 1]
            state["current_strategy"] = next_topic
            next_step = "generate"
            print(f"➡️ [DEBUG] 최대 3회 도달 → 다음 topic으로 이동: {next_topic}")
        else:
            next_step = "end"
            print("🏁 [DEBUG] 모든 topic 완료 → 인터뷰 종료")

    elif is_low:
        next_step = "generate"
        print("🔁 [DEBUG] 최근 평가에 '하' → 같은 topic에서 재질문")

    else:
        if current_index + 1 < len(topic_list):
            next_topic = topic_list[current_index + 1]
            state["current_strategy"] = next_topic
            next_step = "generate"
            print(f"✅ [DEBUG] 성공 응답 → 다음 topic으로 이동: {next_topic}")
        else:
            next_step = "end"
            print("🏁 [DEBUG] 모든 topic 완료 → 인터뷰 종료")

    return {
        **state,
        "next_step": next_step
    }


# 4) 질문 생성 --------------------
def generate_question(state: InterviewState) -> InterviewState:
    llm = ChatOpenAI(model="gpt-4o-mini")
    resume_summary = state.get("resume_summary", "")
    resume_keywords = ", ".join(state.get("resume_keywords", []))
    question_strategy = state.get("question_strategy", {})
    current_strategy = state.get("current_strategy", "")
    stragety = question_strategy[current_strategy]['질문전략']
    current_question = state.get("current_question", "")
    current_answer = state.get("current_answer", "")
    evaluation = state.get("evaluation", [])

    last_evaluation = evaluation[-1] if evaluation else {}

    # 심화(추가) 질문
    prompt = ChatPromptTemplate.from_template("""
당신은 인터뷰 질문을 설계하는 AI입니다.

다음은 추가 질문을 생성하기 위해 참조할 중요한 정보입니다.
- 이력서 요약: {resume_summary}
- 이력서 키워드: {resume_keywords}
- 질문 전략({current_strategy}): {strategy}
- 이전 질문: {current_question}
- 답변: {current_answer}
- 평가: {evaluation}

위 정보를 기반으로 지원자의 사고력, 문제 해결 방식, 혹은 기술적 깊이를 더 확인할 수 있는 심화 인터뷰 질문을 한 가지 생성해주세요.
구체적이고, 지원자의 대답을 확장시킬 수 있는 질문이어야 합니다. 또한 날카로운 질문이어야 합니다. 질문은 한 문장으로 생성합니다.
""")

    formatted_prompt = prompt.format(
        current_question=current_question,
        current_answer=current_answer,
        evaluation=str(last_evaluation),
        resume_summary=resume_summary,
        resume_keywords=resume_keywords,
        strategy=stragety,
        current_strategy=current_strategy
    )

    response = llm.invoke(formatted_prompt)

    return {
        **state,
        "current_question": response.content.strip(),
        "current_answer": ""
    }

# 5) 인터뷰 피드백 보고서 --------------------
import ast
from langchain.schema import HumanMessage

def summarize_interview(state: InterviewState) -> InterviewState:
    print("\n📝 인터뷰 종료 리포트")
    print("=" * 40)

    conversation = state.get("conversation", [])
    evaluations = state.get("evaluation", [])

    for i, (entry, raw_eval) in enumerate(zip(conversation, evaluations)):
        print('\n' + "-" * 30)
        print(f"[📌 질문 내용 {i + 1}] {entry['question']}")
        print(f"[💬 답변 요약 {i + 1}] {entry['answer']}")

        try:
            eval_data = ast.literal_eval(raw_eval)
        except Exception as e:
            print(f"⚠️ 평가 파싱 실패: {e}")
            continue


        topic = eval_data.get("topic", "알 수 없음")
        print(f"[📁 주제] {topic}")
        print("[📊 평가 결과]")
        print()
        per_question_summary = []
        emoji_map = {"상": "🟢", "중": "🟡", "하": "🔴"}

        for key in ["질문과의 연관성", "답변의 구체성", "답변의 길이", "답변의 태도", "문장 완성도"]:
            if key in eval_data:
                등급 = eval_data[key].get("등급", "N/A")
                이유 = eval_data[key].get("이유", "")
                표시 = f"{등급}{emoji_map.get(등급, '')}"
                print(f" - {key}: {등급} ({이유})")
                per_question_summary.append(f"- {key}: {등급} ({이유})")

        # GPT를 통해 per-question 피드백 생성
        eval_summary_text = "\n".join(per_question_summary)
        prompt = f"""
당신은 인사 전문가입니다. 아래는 어떤 지원자가 면접 질문에 대해 답변한 내용을 평가한 결과입니다.

이 평가 내용을 바탕으로, 해당 답변의 강점, 약점, 그리고 개선을 위한 권고사항을 각각 한두 문장씩 간결하게 작성해 주세요.

[평가 내용]
{eval_summary_text}

[응답 형식 예시]
강점: …
약점: …
권고사항: …
"""
        llm = ChatOpenAI(model="gpt-4o-mini")
        gpt_feedback = llm([HumanMessage(content=prompt)]).content.strip()
        print()
        print("[피드백]")
        print(gpt_feedback)

    print("\n✅ 인터뷰를 마쳤습니다.\n")
    return state

# 6) Agent --------------------
# 분기 판단 함수
def route_next(state: InterviewState) -> Literal["generate", "summarize"]:
    return "summarize" if state["next_step"] == "end" else "generate"

# 그래프 정의 시작
builder = StateGraph(InterviewState)

# 노드 추가
builder.add_node("evaluate", evaluate_answer)
builder.add_node("decide", decide_next_step)
builder.add_node("generate", generate_question)
builder.add_node("summarize", summarize_interview)

# 노드 연결
builder.set_entry_point("evaluate")
builder.add_edge("evaluate", "decide")
builder.add_conditional_edges("decide", route_next)
builder.add_edge("generate", END)      # 루프
builder.add_edge("summarize", END)            # 종료

# 컴파일
graph = builder.compile()
#-------------------------------------------------------------------


########### 다음 코드는 제공되는 gradio 코드 입니다.################

import gradio as gr
import tempfile

# 세션 상태 초기화 함수
def initialize_state():
    return {
        "state": None,
        "interview_started": False,
        "interview_ended": False,
        "chat_history": []
    }

# 파일 업로드 후 인터뷰 초기화
def upload_and_initialize(file_obj, session_state):
    if file_obj is None:
        return session_state, "파일을 업로드해주세요."

    # Gradio는 file_obj.name 이 파일 경로야
    file_path = file_obj.name

    # 인터뷰 사전 처리
    state = preProcessing_Interview(file_path)
    session_state["state"] = state
    session_state["interview_started"] = True

    # 첫 질문 저장
    first_question = state["current_question"]
    session_state["chat_history"].append(["🤖 AI 면접관", first_question])

    return session_state, session_state["chat_history"]

# 답변 처리 및 다음 질문 생성
def chat_interview(user_input, session_state):
    if not session_state["interview_started"]:
        return session_state, "먼저 이력서를 업로드하고 인터뷰를 시작하세요."

    # (1) 사용자 답변 저장
    session_state["chat_history"].append(["🙋‍♂️ 지원자", user_input])
    session_state["state"] = update_current_answer(session_state["state"], user_input)

    # (2) Agent 실행 (평가 및 다음 질문 or 종료)
    session_state["state"] = graph.invoke(session_state["state"])

    # (3) 종료 여부 판단
    if session_state["state"]["next_step"] == "end":
        session_state["interview_ended"] = True
        final_summary = "✅ 인터뷰가 종료되었습니다!\n\n"
        for i, turn in enumerate(session_state["state"]["conversation"]):
            final_summary += f"\n**[질문 {i+1}]** {turn['question']}\n**[답변 {i+1}]** {turn['answer']}\n"
            if i < len(session_state["state"]["evaluation"]):
                eval_result = session_state["state"]["evaluation"][i]
                print('✅ ',eval_result)
                print('✅ ',type(eval_result))

                final_summary += eval_result#f"_평가 - 질문 연관성: {eval_result['질문과의 연관성']}, 답변 구체성: {eval_result['답변의 구체성']}_\n"

        session_state["chat_history"].append(["🤖 AI 면접관", final_summary])
        return session_state, session_state["chat_history"], gr.update(value="")

    else:
        next_question = session_state["state"]["current_question"]
        session_state["chat_history"].append(["🤖 AI 면접관", next_question])
        return session_state, session_state["chat_history"], gr.update(value="")

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    session_state = gr.State(initialize_state())

    gr.Markdown("# 🤖 AI 면접관 \n이력서를 업로드하고 인터뷰를 시작하세요!")

    with gr.Row():
        file_input = gr.File(label="이력서 업로드 (PDF 또는 DOCX)")
        upload_btn = gr.Button("인터뷰 시작")

    chatbot = gr.Chatbot()
    user_input = gr.Textbox(show_label=False, placeholder="답변을 입력하고 Enter를 누르세요.")

    upload_btn.click(upload_and_initialize, inputs=[file_input, session_state], outputs=[session_state, chatbot])
    user_input.submit(chat_interview, inputs=[user_input, session_state], outputs=[session_state, chatbot])
    user_input.submit(lambda: "", None, user_input)

# 실행
demo.launch(share=True)

Overwriting app.py


## **3. 실행**

In [14]:
!python app.py

/content/app.py:587: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://65f22cde896c455854.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
/content/app.py:283: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([HumanMessage(content=prompt_text)]).content
✅ [DEBUG] 성공 응답 → 다음 topic으로 이동: 동기 및 커뮤니케이션
/usr/local/lib/python3.11/dist-packages/